In [7]:
import pandas as pd
import numpy as np
import sklearn



In [28]:
data_essays = pd.read_csv('data/train_essays.csv')
data_essays_test = pd.read_csv('data/test_essays.csv')


In [26]:
data_essays

In [24]:
import numpy as np
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [37]:
# Предполагается, что у вас есть данные, содержащие тексты и соответствующих им авторов.
# Предполагается, что вы уже предобработали тексты и собрали их в формате списка.

# Пример данных
texts = data_essays['text'].tolist()
authors = data_essays['generated'].tolist()

# # Используйте LabelEncoder для преобразования меток авторов в числа
# label_encoder = LabelEncoder()
# encoded_authors = label_encoder.fit_transform(authors)

# Обучите Word2Vec модель на ваших текстах (здесь приведен пример)
# Мы предполагаем, что тексты были предварительно обработаны (токенизированы, удалены стоп-слова и т.д.)
word2vec_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)

# Получите векторы для каждого текста, усредняя векторы его слов
def get_text_vector(text):
    vectors = [word2vec_model.wv[word] for word in text.split() if word in word2vec_model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Преобразуйте все тексты в векторы
text_vectors = [get_text_vector(text) for text in texts]

# Разделите данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(text_vectors, authors, test_size=0.2, random_state=42)

# Создайте модель нейронной сети
model = Sequential()
model.add(Dense(64, input_dim=word2vec_model.vector_size, activation='relu'))
model.add(Dense(64, input_dim=word2vec_model.vector_size, activation='relu'))
model.add(Dense(len(np.unique(authors)), activation='softmax'))

# Скомпилируйте модель
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучите модель
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=32, validation_split=0.2)

# Оцените модель
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Loss: {loss}, Accuracy: {accuracy}')

In [43]:
model.predict(np.array(X_test))

In [44]:
X_test

In [ ]:
/